# "Local affine features: useful side product"
> "Why to throw away an important information?"
- toc: false
- image: images/affine_matches.png
- branch: master
- badges: true
- comments: true
- hide: false
- search_exclude: false

## Keypoints are not just points

![image.png](2020-07-17-affine-correspondences_files/att_00000.png)

Wide baseline stereo matching often as perceived as establishing (key-)point correspondences between images. While this might be true for the some local features like SuperPoint \cite{SuperPoint2017}, typically it is more than that. 

Specifically, detectors like DoG\cite{Lowe99}, Harris\cite{Harris88}, Hessian\cite{Hessian78}, KeyNet\cite{KeyNet2019}, ORB\cite{ORB2011}, and many others rate on scale-space provide at least 3 parameters: x, y, and scale.  

Most of the local descriptors  -- SIFT\cite{Lowe99}, HardNet\cite{HardNet2017} and so on -- are not rotation invariant and those which are - mostly require complex matching function\cite{RIFT2005}, \cite{sGLOH2}, so the patch orientation has to be estimated anyway, in order to match reliably. This can be done by various methods:
corners center of mass (ORB\cite{ORB2011}, dominant gradient orientation (SIFT)\cite{Lowe99} or by some learned estimator (OriNets\cite{OriNet2016},\cite{AffNet2018}). Sometimes it is possible to rely on smartphone camera IMU or photographer and assume that images are upright\cite{PerdochRetrieval2009}.

Thus, we can assume that if local descriptors match, this means the local feature scale and orientation also match, at least approximately -- see Figure below. Possible exceptions are cases, when the patch is symmetrical and orientation is ambiguous up to some symmetry group.


![](2020-07-17-affine-correspondences_files/matches_patches.png "Selected matching SIFT keypoints and corresponding patches. One can see that not only patch centers correspond to each other, but also other pixels, although less precise. Image pair from Sacre Coeur IMW dataset.")


In addition, one could assume that we observe the patch not from the fronto-parallel position and try to estimate local normal, or, more precisely, affine shape of the feature point, modeling it as an ellipse instead of circle. One could also think of affine shape estimation as finding the camera position, from where the patch is seen in some "canonical" view.

![](2020-07-17-affine-correspondences_files/affinematches_patches.png "Selected matching SIFT-AffNet keypoints and corresponding patches. One can see that not only patch centers correspond to each other, but also other pixels, although less precise. Image pair from Sacre Coeur IMW dataset.")

This gives us 3 points correspondences from a single local feature match, see an example in Figure below.

![](2020-07-17-affine-correspondences_files/laf-check-illustration.png "Local affine correspondences. While centers of both regions A and B are correct point matches, only A is a correct affine correspondence.")




Why is it important and how to use it -- see in current post. How to esimate local affine features robustly -- in the next post.

# Benefits of local affine features

## Making descriptor job easier

The most straightforward benefit of using local affine features is that they increase the repeatability of the detector and potentially reduce appearance changes of a local patch caused by viewpoint difference. This makes possible matching more challenging image pairs.

![](2020-07-17-affine-correspondences_files/test.png "Hessian features + HardNet matches + RANSAC inliers, Right: HessAffNet features + HardNet matches + RANSAC inliers. Image pair from Tanks & Temples. Epipolar lines are shown in cyan.")


![](2020-07-17-affine-correspondences_files/att_00009.png "Repeatability and the number of correspondences. AffNet compared with the de facto standard Baumberg iteration according to the Mikolajczyk protocol. Left – images with illumination differences, right – with viewpoint and scale changes. SS – patch from the scale-space pyramid at the level of the detection, image – from the original image; 19 and 33 are patch sizes.") 


The practice is a little bit more complicated. Our recent benchmark\cite{IMW2020}, which measure the accuracy of the output fundamental matrix, shows that the difference in using affine and similarity-covariant features is quite minor. Specifically, the relative difference between SIFT vs SIFT-Affine features is 0.5% and between Hessian-SIFT and Hessian-AffNet SIFT is 5.1%, see Table below.

![](2020-07-17-affine-correspondences_files/att_00011.png "Test – Stereo results with 8k features. We report: NF --  Number of Features; NI -- Number of Inliers produced by RANSAC; and mAA@10°.")

Therefore, if the benefit of local features would be to only improve descriptor extraction stage, it would be arguably not worth it. Luckily, there are more benefits, which are more pronounced.


## Making RANSAC job easier

Let's recall how RANSAC works. 

1. Randomly sample a minimally required number of tentative correspondences to fit the geometrical model of the scene: 4 for homography, 7 for epipolar geometry and estimate the model.
2. Calculate "support": other correspondeces, which are consistent with the model. 
3. Repeat steps (1), (2) and output the model which is supported with the most of correspondences. If you were lucky and have sampled all-inlier sample, meaning that all correspondences used to estimate the model were correct, you would have a correct model.


Reality is more complicated than I have just described, but the principle is the same. 
The most important part is the sampling and it is sensitive to inlier ratio $\nu$ - the percentage of the correct correspondences in the set. Lets denote the minimal number of correspondences required to estimate the model as **m**.
To recover the correct model with the confidence **p** one needs to sample the number of correspondences, which is described by formula:

\begin{equation}
N = \frac{\log{(1 - p)}}{\log{(1 - \nu^{m})}}
\end{equation}

Lets plot the how the number of required samples changes with inlier ratio for confidence equal 99%. Note the log scale on Y axis. Different lines are for different minimal sample size **m**.

![](2020-07-17-affine-correspondences_files/att_00016.png "Number of samples to find correct model as a function of inlier ratio.")

As you can see from the plot above, reducing the minimal sample size required for the model estimation even by 1 saves and order of magnitude of computation. 
In reality the benefit is a smaller, as modern RANSACs like GC-RANSAC\cite{gcransac2018} and MAGSAC\cite{magsac2019} could estimate the correct model from the sample containing outliers, but it is still huge, especially for low inlier rate cases.


### Image retrieval
The ideal case would be to estimate a model from just a single sample and that is exactly what is done in spatial reranking paper "**Object retrieval with large vocabularies and fast spatial matching**"  by Philbin et.al \cite{Philbin07}.

Specifically, they are solving a particular object retrieval problem: given an image containing some object, return all the images from the database, which also containg the same object.

The inital list of images is formed by the descriptor distance and then is reranked.
The authors propose to approximate a perspective change between two images as an affine image transformation, and count number of feature points, which are reprojected inside the second image. This number produces better ranking that the original short-list. 

![](2020-07-17-affine-correspondences_files/att_00012.png "Figure from  Philbin et.al")

### Back to wide baseline stereo

While working for spatial re-ranking, 3-degrees of freedom camera model is too rough for the wide baseline stereo.
Yet, going from 4 point correspondences (PC) to 2 affine correspondences (AC) for homogaphy and from 7 PC to 3 AC for the fundamental matrix would be huge benefit anyway for the robust model estimation. 

Various variant of RANSAC working for local features were proposed in the last 15 years: Perdoch et.al\cite{perd2006epipolar}, Pritts et.al.\cite{PrittsRANSAC2013}, Barath and Kukelova \cite{Barath2019ICCV}, Rodríguez et.al\cite{RANSACAffine2020}.

Finally, the systematic study of using is presented by Barath et.al\cite{barath2020making} in "Making Affine Correspondences Work in Camera Geometry Computation" paper. Authors show that if used naively, affine correspondence lead to worse results, because they are more noisy than point correspondences. 
However, there is a bag of tricks presented in the paper, which allow to solve the noise issue and make the affine RANSAC working in practice, resulting in orders of magnitude faster computation.

![](2020-07-17-affine-correspondences_files/att_00013.png "Figure from Making Affine Correspondences Work in Camera Geometry Computation")

Moreover, for a special cases like autonomous driving, where the motion is mostly horizonal, one could even use 2 affine correspondes for both motion estimation and consistency check, significantly improving the efficiency of the outliers removal compared to the standard RANSAC loop\cite{guan2020relative}. 

![](2020-07-17-affine-correspondences_files/att_00001.png "Empirical cumulative error distributions for KITTI sequence 00. Figure from Relative Pose Estimation for Multi-Camera Systems from Affine Correspondences.")

Besides the special case considerations, additional contraints can also come from running other algorithms, like monocular depth estimation. Such a constraint could reduce the required number of matches from two affine correspondences to a single one for calibrated camera case\cite{OneACMonoDepth2020}.

![](2020-07-17-affine-correspondences_files/att_00002.png "Figure from Relative Pose from Deep Learned Depth and a Single Affine Correspondence")


![](2020-07-17-affine-correspondences_files/att_00003.png "Figure from Relative Pose from Deep Learned Depth and a Single Affine Correspondence")

## Application-specific benefits

Besides the wide baseline stereo, local affine features and correspondences have other applications. I will briefly describe some of them here (to be updated).

### Image rectification


Instead of matching local features between two images one might match them within a single image. Why would someone do it? This allows finding repeated pattern: think about windows, doors and so on. Typically they have the same physical size, therefore the diffrence in local features around them could tell us about the geometry of the scene and lens distortion. 

![](2020-07-17-affine-correspondences_files/att_00008.png "Repeated patterns detection with MSER andRootSIFT local features. Figure from the Minimal Solvers for Rectifying from Radially-Distorted Conjugate Translations, PAMI 2020 paper.")


This is the idea of the series of works by Pritts and Chum. 

![](2020-07-17-affine-correspondences_files/att_00007.png "Figure from the 'Minimal Solvers for Rectifying from Radially-Distorted Conjugate Translations', PAMI 2020 paper.")

### Surface normals estimation

Ivan Eichhardt and Levente Hajder have a series of works, exploiting the local affine correspondences for surface normals estimation\cite{SurfaceNormals2019}


![](2020-07-17-affine-correspondences_files/att_00015.png "Estimated surface normals. Figure from Optimal Multi-View Surface Normal Estimation Using Affine Correspondences")


# Summary

Despite not being popular right now, treating keypoints as local affine features has a lot of advantages over the traditional treatment the local correspondence as the point correspondences. In the next post I will describe a way of estimating the local feature affine shape and orientation.

# References

[<a id="cit-SuperPoint2017" href="#call-SuperPoint2017">SuperPoint2017</a>] Detone D., Malisiewicz T. and Rabinovich A., ``_Superpoint: Self-Supervised Interest Point Detection and Description_'', CVPRW Deep Learning for Visual SLAM, vol. , number , pp. ,  2018.

[<a id="cit-Lowe99" href="#call-Lowe99">Lowe99</a>] D. Lowe, ``_Object Recognition from Local Scale-Invariant Features_'', ICCV,  1999.

[<a id="cit-Harris88" href="#call-Harris88">Harris88</a>] C. Harris and M. Stephens, ``_A Combined Corner and Edge Detector_'', Fourth Alvey Vision Conference,  1988.

[<a id="cit-Hessian78" href="#call-Hessian78">Hessian78</a>] P.R. Beaudet, ``_Rotationally invariant image operators_'', Proceedings of the 4th International Joint Conference on Pattern Recognition,  1978.

[<a id="cit-KeyNet2019" href="#call-KeyNet2019">KeyNet2019</a>] A. Barroso-Laguna, E. Riba, D. Ponsa <em>et al.</em>, ``_Key.Net: Keypoint Detection by Handcrafted and Learned CNN Filters_'', ICCV,  2019.

[<a id="cit-ORB2011" href="#call-ORB2011">ORB2011</a>] E. Rublee, V. Rabaud, K. Konolidge <em>et al.</em>, ``_ORB: An Efficient Alternative to SIFT or SURF_'', ICCV,  2011.

[<a id="cit-HardNet2017" href="#call-HardNet2017">HardNet2017</a>] A. Mishchuk, D. Mishkin, F. Radenovic <em>et al.</em>, ``_Working Hard to Know Your Neighbor's Margins: Local Descriptor Learning Loss_'', NeurIPS,  2017.

[<a id="cit-RIFT2005" href="#call-RIFT2005">RIFT2005</a>] {Lazebnik} S., {Schmid} C. and {Ponce} J., ``_A sparse texture representation using local affine regions_'', IEEE Transactions on Pattern Analysis and Machine Intelligence, vol. 27, number 8, pp. 1265-1278,  2005.

[<a id="cit-sGLOH2" href="#call-sGLOH2">sGLOH2</a>] {Bellavia} F. and {Colombo} C., ``_Rethinking the sGLOH Descriptor_'', IEEE Transactions on Pattern Analysis and Machine Intelligence, vol. 40, number 4, pp. 931-944,  2018.

[<a id="cit-OriNet2016" href="#call-OriNet2016">OriNet2016</a>] K. M., Y. Verdie, P. Fua <em>et al.</em>, ``_Learning to Assign Orientations to Feature Points_'', CVPR,  2016.

[<a id="cit-AffNet2018" href="#call-AffNet2018">AffNet2018</a>] D. Mishkin, F. Radenovic and J. Matas, ``_Repeatability is Not Enough: Learning Affine Regions via Discriminability_'', ECCV,  2018.

[<a id="cit-PerdochRetrieval2009" href="#call-PerdochRetrieval2009">PerdochRetrieval2009</a>] M. {Perd'och}, O. {Chum} and J. {Matas}, ``_Efficient representation of local geometry for large scale object retrieval_'', CVPR,  2009.

[<a id="cit-IMW2020" href="#call-IMW2020">IMW2020</a>] Jin Yuhe, Mishkin Dmytro, Mishchuk Anastasiia <em>et al.</em>, ``_Image Matching across Wide Baselines: From Paper to Practice_'', arXiv preprint arXiv:2003.01587, vol. , number , pp. ,  2020.

[<a id="cit-gcransac2018" href="#call-gcransac2018">gcransac2018</a>] D. Barath and J. Matas, ``_Graph-Cut RANSAC_'', The IEEE Conference on Computer Vision and Pattern Recognition (CVPR),  2018.

[<a id="cit-magsac2019" href="#call-magsac2019">magsac2019</a>] J.N. Daniel Barath, ``_MAGSAC: marginalizing sample consensus_'', CVPR,  2019.

[<a id="cit-Philbin07" href="#call-Philbin07">Philbin07</a>] J. Philbin, O. Chum, M. Isard <em>et al.</em>, ``_Object Retrieval with Large Vocabularies and Fast Spatial Matching_'', CVPR,  2007.

[<a id="cit-perd2006epipolar" href="#call-perd2006epipolar">perd2006epipolar</a>] M. Perd'och, J. Matas and O. Chum, ``_Epipolar geometry from two correspondences_'', ICPR,  2006.

[<a id="cit-PrittsRANSAC2013" href="#call-PrittsRANSAC2013">PrittsRANSAC2013</a>] J. {Pritts}, O. {Chum} and J. {Matas}, ``_Approximate models for fast and accurate epipolar geometry estimation_'', 2013 28th International Conference on Image and Vision Computing New Zealand (IVCNZ 2013),  2013.

[<a id="cit-Barath2019ICCV" href="#call-Barath2019ICCV">Barath2019ICCV</a>] D. Barath and Z. Kukelova, ``_Homography From Two Orientation- and Scale-Covariant Features_'', ICCV,  2019.

[<a id="cit-RANSACAffine2020" href="#call-RANSACAffine2020">RANSACAffine2020</a>] M. {Rodríguez}, G. {Facciolo}, R. G. <em>et al.</em>, ``_Robust estimation of local affine maps and its applications to image matching_'', 2020 IEEE Winter Conference on Applications of Computer Vision (WACV),  2020.

[<a id="cit-barath2020making" href="#call-barath2020making">barath2020making</a>] Barath Daniel, Polic Michal, Förstner Wolfgang <em>et al.</em>, ``_Making Affine Correspondences Work in Camera Geometry Computation_'', arXiv preprint arXiv:2007.10032, vol. , number , pp. ,  2020.

[<a id="cit-guan2020relative" href="#call-guan2020relative">guan2020relative</a>] Guan Banglei, Zhao Ji, Barath Daniel <em>et al.</em>, ``_Relative Pose Estimation for Multi-Camera Systems from Affine Correspondences_'', arXiv preprint arXiv:2007.10700, vol. , number , pp. ,  2020.

[<a id="cit-OneACMonoDepth2020" href="#call-OneACMonoDepth2020">OneACMonoDepth2020</a>] D.B. Ivan Eichhardt, ``_Relative Pose from Deep Learned Depth and a Single Affine Correspondence_'', ECCV,  2020.

[<a id="cit-SurfaceNormals2019" href="#call-SurfaceNormals2019">SurfaceNormals2019</a>] {Baráth} D., {Eichhardt} I. and {Hajder} L., ``_Optimal Multi-View Surface Normal Estimation Using Affine Correspondences_'', IEEE Transactions on Image Processing, vol. 28, number 7, pp. 3301-3311,  2019.

